<a href="https://colab.research.google.com/github/wayne-chi/Cassava_disease_detection_using_ensemble_models/blob/main/Squeezenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [ ]:
name_output = input('enter the name of your output file ').strip()

enter the name of your output file squeezenetfirst



# Import Modules

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Reshape, Activation,Flatten,BatchNormalization,Conv2D,MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Recall, Precision,TruePositives,TrueNegatives, FalsePositives,FalseNegatives,Accuracy

In [ ]:
import os
import shutil
import glob
import random
import warnings
# warnings.simplefilter(action='ignore',category='FutureWarning')

In [ ]:
print('hello world')


hello world


# Add GPU

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print('Num GPUs Available: ', len(physical_devices))
if len(physical_devices) >=1:
    tf.config.experimental.set_memory_growth(physical_devices[0],True)

Num GPUs Available:  1


# Make your edits

In [ ]:
this_model_name = name_output+'.h5'

In [ ]:
model_name='Mobilenet_3'

In [ ]:
image_zip = 'All_cassava_data_2.zip'   #'More_Cassava.zip'#'clean_annotated_cassava.zip'

In [ ]:
## chnage home directory
home = '/content/drive/MyDrive/project/Project2.1/Script2_pure classification' #'/content/drive/MyDrive/project/Tensorflow And Keras' #

# Pull Data and Setup

In [ ]:


image_path = model_name + '_data'
os.mkdir(image_path)

shutil.copy(os.path.join(home,'Images',image_zip),image_zip)

'All_cassava_data_2.zip'

In [ ]:
shutil.unpack_archive(image_zip,os.getcwd(),'zip')

In [ ]:
folders = os.listdir(image_zip[:-4])
folders

['brownstreak_N', 'Mosaic_N', 'greenmite_N', 'blight_N']

### Edit here or type name of folder

In [ ]:
f_folders = list(filter(lambda x:  x.endswith('N') , folders) )
f_folders = sorted(f_folders)
f_folders

['Mosaic_N', 'blight_N', 'brownstreak_N', 'greenmite_N']

## Create path and arrange

In [ ]:
os.mkdir(os.path.join(image_path,'train'))
os.mkdir(os.path.join(image_path,'test'))
os.mkdir(os.path.join(image_path,'valid'))

In [ ]:
for fold in f_folders:
  src = os.path.join(image_zip[:-4],fold)
  dest = os.path.join(image_path,'train')
  shutil.move(src,dest)

  c_test_dis = os.path.join(image_path,'test',fold)
  c_valid_dis = os.path.join(image_path,'valid',fold)
  os.mkdir(c_test_dis)
  os.mkdir(c_valid_dis)

## delete xml files
  total_files = os.listdir(os.path.join(dest,fold))
  to_del = filter(lambda x: x.endswith('xml'),total_files)
  for i in to_del:
    os.remove(os.path.join(dest,fold,i))

## pick out a few and move to valid
  valid_amount = 2 * len(os.listdir(os.path.join(dest,fold)))  //10
  test_amount = len(os.listdir(os.path.join(dest,fold))) //10

  valid_samples = random.sample(os.listdir(os.path.join(dest,fold)), valid_amount)
  for f in valid_samples:
        shutil.move(os.path.join(dest,fold,f), c_valid_dis)

## pick out a few and move to test
  test_samples = random.sample(os.listdir(os.path.join(dest,fold)), test_amount)
  for k in test_samples:
        shutil.move(os.path.join(dest,fold,k), c_test_dis)

# The Plot Functions

## Plot Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm,classes,
                          normalize = False,
                          title='Confusion matrix',
                          cmap = plt.cm.Blues):
  """
  this function prints and plots the confusion matrix
  normalization can be applied by setting normalize=True
  """
  plt.imshow(cm,interpolation='nearest',cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tickmarks=  np.arange(len(classes))
  plt.xticks(tickmarks,classes,rotation=45)
  plt.yticks(tickmarks,classes)
  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1) [:,np.newaxis]
    print('Normalized confusion matrix')
  else:
    print('confusion matrix without normalization')

  thresh = cm.max()/2
  for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
    plt.text(i,j,cm[i,j],
             horizontalalignment='center',
             color = 'white' if cm[i,j] > thresh else 'black'
             )
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('predicted label')

# Instantiate the model

In [ ]:
pip install keras-contrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement keras-contrib (from versions: none)
ERROR: No matching distribution found for keras-contrib


In [ ]:
from keras_contrib.applications import squeezeNet


In [ ]:
!pip install keras_squeezenet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://github.com/rcmalli/keras-squeezenet.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/rcmalli/keras-squeezenet.git to /tmp/pip-req-build-9sh83xc7
  Running command git clone --filter=blob:none --quiet https://github.com/rcmalli/keras-squeezenet.git /tmp/pip-req-build-9sh83xc7
  Resolved https://github.com/rcmalli/keras-squeezenet.git to commit 4fb9cb7510ea0315303090edbc1bd97c2916af81
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install keras --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip uninstall tensorflow keras
!pip install tensorflow==2.2.0 keras

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? y
Y
  Successfully uninstalled tensorflow-2.12.0
Found existing installation: keras 2.12.0
Uninstalling keras-2.12.0:
  Would remove:
    /usr/local/lib/python3.9/dist-packages/keras-2.12.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/keras/*
Proceed (Y/n)?   Successfully uninstalled keras-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow

In [ ]:
pip install https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow-1.4.0-cp35-cp35m-linux_x86_64.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: tensorflow-1.4.0-cp35-cp35m-linux_x86_64.whl is not a supported wheel on this platform.


In [ ]:
from  keras_squeezenet import squeezenet

ImportError: ignored

In [ ]:
Home_model = from keras_squeezenet import squeezenets.SqueezeNet(weights ='imagenet', include_top=False)



AttributeError: ignored

In [ ]:
import tensorflow as tf

# Load SqueezeNet without the last layer
base_model = tf.keras.applications.SqueezeNet(weights='imagenet', include_top=False)

# Add new output layer for your task
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

# Create new model with modified output layer
model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False


## Preprocess

In [ ]:
from tensorflow.keras.preprocessing import image

def prepare_image(file):
  img = image.load_img(file, target_size = (224,224))
  img_array = image.img_to_array(img)
  # print(img_array.shape)
  img_array_expanded = np.expand_dims(img_array,axis=0)
  # print(img_array_expanded.shape)
  return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded)

## Transfer Learning

In [ ]:
train_path = os.path.join(image_path,'train')
valid_path = os.path.join(image_path,'valid')
test_path = os.path.join(image_path,'test')

In [ ]:
from keras.utils import to_categorical

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
              .flow_from_directory(directory= train_path,target_size=(224,224),batch_size=10, class_mode='categorical')

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
              .flow_from_directory(directory= valid_path,target_size=(224,224),batch_size=10, class_mode='categorical')

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)\
              .flow_from_directory(directory= test_path,target_size=(224,224),batch_size=10, shuffle = False, class_mode='categorical')


Found 3663 images belonging to 4 classes.
Found 1043 images belonging to 4 classes.
Found 520 images belonging to 4 classes.


, class_mode='binary'





In [ ]:
def data_count(batch):
  classes_dict = {v:k for k,v in batch.class_indices.items()}
  train_y = pd.DataFrame(np.concatenate([batch.next()[1] for i in range(batch.__len__())]) )
  train_y.rename(classes_dict,axis=1,inplace=True)
  return train_y.sum(axis=0).astype('int')

In [ ]:
data_count(train_batches)

Mosaic_N          120
blight_N         1120
brownstreak_N    1686
greenmite_N       737
dtype: int64

In [ ]:
data_count(valid_batches)

Mosaic_N          33
blight_N         319
brownstreak_N    481
greenmite_N      210
dtype: int64

In [ ]:
data_count(test_batches)

Mosaic_N          16
blight_N         159
brownstreak_N    240
greenmite_N      105
dtype: int64

In [ ]:
Home_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [ ]:
len(Home_model.layers)

91

In [ ]:
x = Home_model.layers[-5].output
s2l = Reshape((-1,))(x)
s2l = Model(inputs= Home_model.input,outputs = s2l)
output = Dense(units=4,activation='sigmoid')(s2l.output)  ## sigmoid softmax


In [ ]:
model = Model(inputs= Home_model.input,outputs = output)

In [ ]:
for layer in  model.layers[:-23]:
  layer.trainable=False

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

# Train The Model

In [ ]:
model.compile(
    optimizer =Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy',Recall(), Precision(),TruePositives(),TrueNegatives(), FalsePositives(),FalseNegatives()]
)

In [ ]:
epochs=  20

In [ ]:
train_labels = to_categorical(train_batches.classes)
valid_labels = to_categorical(valid_batches.classes)
train_labels

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [ ]:
type(train_batches)

keras.preprocessing.image.DirectoryIterator

In [ ]:
History = model.fit(
    train_batches,

    validation_data = valid_batches,
    epochs = epochs,
    verbose=2,

    )

Epoch 1/20
367/367 - 59s - loss: 0.4069 - accuracy: 0.5635 - recall_1: 0.4682 - precision_1: 0.5812 - true_positives_1: 1715.0000 - true_negatives_1: 9753.0000 - false_positives_1: 1236.0000 - false_negatives_1: 1948.0000 - val_loss: 0.5378 - val_accuracy: 0.4036 - val_recall_1: 0.3030 - val_precision_1: 0.4125 - val_true_positives_1: 316.0000 - val_true_negatives_1: 2679.0000 - val_false_positives_1: 450.0000 - val_false_negatives_1: 727.0000 - 59s/epoch - 161ms/step
Epoch 2/20
367/367 - 58s - loss: 0.3343 - accuracy: 0.6467 - recall_1: 0.5790 - precision_1: 0.6729 - true_positives_1: 2121.0000 - true_negatives_1: 9958.0000 - false_positives_1: 1031.0000 - false_negatives_1: 1542.0000 - val_loss: 0.5912 - val_accuracy: 0.3806 - val_recall_1: 0.3250 - val_precision_1: 0.3800 - val_true_positives_1: 339.0000 - val_true_negatives_1: 2576.0000 - val_false_positives_1: 553.0000 - val_false_negatives_1: 704.0000 - 58s/epoch - 159ms/step
Epoch 3/20
367/367 - 53s - loss: 0.3108 - accuracy: 0.

In [ ]:
History.history

{'loss': [1.1601451635360718,
  0.903500497341156,
  0.8029628992080688,
  0.7663958072662354,
  0.7290693521499634,
  0.722443699836731,
  0.706504225730896,
  0.7011103630065918,
  0.6916917562484741,
  0.6669007539749146,
  0.6613993048667908,
  0.6614077687263489,
  0.6520191431045532,
  0.6271476745605469,
  0.6453755497932434,
  0.6307166814804077,
  0.6159552931785583,
  0.6171227693557739,
  0.6072274446487427,
  0.6089116930961609],
 'accuracy': [0.4572754502296448,
  0.5806716084480286,
  0.6224406361579895,
  0.6238056421279907,
  0.642642617225647,
  0.647283673286438,
  0.6461916565895081,
  0.6440076231956482,
  0.647283673286438,
  0.6587496399879456,
  0.664755642414093,
  0.6560196280479431,
  0.6516516804695129,
  0.669396698474884,
  0.6601146459579468,
  0.6590226292610168,
  0.6609336733818054,
  0.661479651927948,
  0.6655746698379517,
  0.6644826531410217],
 'recall': [0.3808353841304779,
  0.5233415365219116,
  0.5700245499610901,
  0.5872235894203186,
  0.60660

In [ ]:
import pandas as pd

In [ ]:
history_df = pd.DataFrame(History.history)

In [ ]:
history_df.to_csv(os.path.join(home,'output',name_output+'.csv'))

In [ ]:
this_model_name

'April_11.h5'

## Load Model

In [ ]:
model_path = os.path.join(home,'models',this_model_name)
# home = '/content/drive/MyDrive/project/Tensorflow And Keras' #/content/drive/MyDrive/project/Project2.1/Script2_pure classification

In [ ]:
loaded_model = keras.models.load_model(model_path)

In [ ]:
model = loaded_model

# Predict

In [ ]:
img = "/content/train-cb_cgm- (3).jpg"
img = prepare_image(img)
# img

In [ ]:
classes_dict = {v:k for k,v in test_batches.class_indices.items()}
classes_dict

{0: 'Mosaic_N', 1: 'blight_N', 2: 'brownstreak_N', 3: 'greenmite_N'}

In [ ]:
pred = model.predict(img)
probs = pred[0].tolist()

# Print the disease probabilities
print(probs)

1/1 [==============================] - 0s 35ms/step
[0.01721510849893093, 0.5098218321800232, 0.0314774289727211, 0.6906486749649048]


In [ ]:
test_labels= test_batches.classes

In [ ]:
x_test =  np.concatenate([test_batches.next()[0] for i in range(test_batches.__len__())])

In [ ]:
predictions = model.predict(  x = x_test,verbose=0)

In [ ]:
dff = pd.DataFrame(predictions)

In [ ]:
dff['test'] = test_labels
dff.head(10)

,0,1,2,3,test
0,9.998465e-01,1.769902e-07,0.000143,1.001612e-05,0
1,9.996089e-01,4.975902e-07,0.000383,7.537441e-06,0
2,9.991946e-01,5.965278e-09,0.000805,4.685008e-08,0
3,9.994639e-01,5.899682e-07,0.000532,3.052613e-06,0
4,9.998662e-01,7.409365e-07,0.000005,1.280910e-04,0
5,9.981810e-01,8.950511e-07,0.000029,1.789128e-03,0
6,9.989567e-01,3.682499e-05,0.000102,9.048400e-04,0
7,9.998772e-01,6.791831e-07,0.000024,9.835656e-05,0
8,9.437779e-07,9.999617e-01,0.000019,1.841862e-05,1
9,2.262162e-06,9.997405e-01,0.000124,1.329854e-04,1


### Take prediction probabilities to a csv file

In [ ]:
dff.to_csv(os.path.join(home,'output',name_output+'_prediction_probabilities.csv'))

### Confusion Matrix

In [ ]:
cm = confusion_matrix(y_pred = np.argmax(predictions,axis=-1),y_true=test_labels)

In [ ]:
conf_mat = pd.DataFrame(cm)
classes_dict = {v:k for k,v in test_batches.class_indices.items()}
conf_mat.rename(classes_dict,inplace=True,axis=0)
conf_mat.rename(classes_dict,inplace=True,axis=1)
conf_mat

,Mosaic_N,blight_N,brownstreak_N,greenmite_N
Mosaic_N,8,0,0,0
blight_N,0,38,1,1
brownstreak_N,0,0,86,1
greenmite_N,0,0,0,35


### Take confusion matrix to a csv file

In [ ]:
conf_mat.to_csv(os.path.join(home,'output',name_output+'_confusion_matix.csv'))


In [ ]:
test_batches.class_indices

In [ ]:
cm_plot_labels = [i for i in test_batches.class_indices.keys()]

In [ ]:
plot_confusion_matrix(cm=cm,classes=cm_plot_labels,title = 'confusion matrix')

### piclkle the variables

In [ ]:
## piclkle the variables
import pickle

In [ ]:
pck_loc = os.path.join(home,'output','classes_dict'+'.pkl')

In [ ]:
with open(pck_loc, 'wb') as file:

    # A new file will be created
    pickle.dump(classes_dict, file)

# Save Model

In [ ]:
model.save(this_model_name)

In [ ]:
model_path = os.path.join(home,'models')
# home = '/content/drive/MyDrive/project/Tensorflow And Keras' #/content/drive/MyDrive/project/Project2.1/Script2_pure classification

In [ ]:
shutil.move(this_model_name,model_path)

'/content/drive/MyDrive/project/Project2.1/Script2_pure classification/models/April_11.h5'